In [2]:
"""
===========================================================
YSI Feeding Calculator
Autor: ChatGPT

Descripción:
    - Lee archivo procesado y en formato ancho (ysi_summary_wide.csv)
    - Pregunta al usuario unidades de cada metabolito
    - Pregunta concentración del stock y volumen de cultivo
    - Calcula volumen de alimento necesario por pozo
    - Guarda archivo "ysi_feeding_plan.csv"

Entrada esperada:
    PlateSequenceName | WellId | Glucose | Lactate | Glutamine | Glutamate

===========================================================
"""

import pandas as pd
import tkinter as tk
from tkinter import filedialog

def ask_file():
    root = tk.Tk()
    root.withdraw()
    filepath = filedialog.askopenfilename(
        title="Selecciona el archivo YSI en formato ancho",
        filetypes=[("CSV Files", "*.csv"), ("All files", "*.*")]
    )
    return filepath


def ask_float(prompt, default):
    """Pide un número al usuario con default."""
    val = input(f"{prompt} [{default}]: ").strip()
    return float(val) if val else default


def main():

    print("\n=== Selecciona el archivo con formato ancho ===")
    filepath = ask_file()

    if not filepath:
        print("No seleccionaste archivo. Abortando.")
        return

    df = pd.read_csv(filepath)

    print("\n=== Datos cargados correctamente ===")
    print(df.head(), "\n")

    # ------------------------------
    # Preguntas al usuario
    # ------------------------------

    # Unidades reales en el archivo
    glucose_units = input("¿En qué unidades está la glucosa? (g/L, mM, etc.) [g/L]: ") or "g/L"
    glutamine_units = input("¿En qué unidades está la glutamina? (mM, mmol/L, g/L) [mM]: ") or "mM"

    # Stock concentrations
    stock_glc = ask_float("Concentración del stock de glucosa (g/L)", 200)
    stock_gln = ask_float("Concentración del stock de glutamina (mM)", 200)

    # Volumen del pozo
    vol_cultivo = ask_float("Volumen del cultivo en cada pozo (mL)", 25)

    # Target concentrations (preguntar)
    target_glc = ask_float("¿A qué concentración quieres llevar la glucosa? (g/L)", 6.0)
    target_gln = ask_float("¿A qué concentración quieres llevar la glutamina? (mM)", 6.0)

    # ------------------------------
    # Conversión si fuera necesario
    # (Solo si el usuario puso unidades distintas)
    # ------------------------------

    # Placeholder: aquí podríamos implementar conversiones
    # de masa↔moles si lo necesitas en el futuro.
    # Ahora asumimos que usuario da unidades coherentes.


    # ------------------------------
    # Cálculo de volúmenes
    # ------------------------------
    df["Feed_Glucose_mL"] = ((target_glc - df["Glucose"]) * vol_cultivo) / stock_glc
    df["Feed_Glutamine_mL"] = ((target_gln - df["Glutamine"]) * vol_cultivo) / stock_gln

    # Evitar valores negativos (si ya está arriba del target)
    df["Feed_Glucose_mL"] = df["Feed_Glucose_mL"].clip(lower=0)
    df["Feed_Glutamine_mL"] = df["Feed_Glutamine_mL"].clip(lower=0)

    # Guardar archivo
    output = "ysi_feeding_plan.csv"
    df.to_csv(output, index=False)

    print("\n=== Archivo generado ===")
    print(output)
    print("\nEjemplo:\n")
    print(df.head())



if __name__ == "__main__":
    main()



=== Selecciona el archivo con formato ancho ===

=== Datos cargados correctamente ===
    PlateSequenceName   WellId   Glucose   Lactate  Glutamine  Glutamate
0  20251114-EBT-T0-T3  R24_A01  5.525105  0.406734   5.203998   2.037303
1  20251114-EBT-T0-T3  R24_A02  5.728371  0.465841   5.241306   1.911974
2  20251114-EBT-T0-T3  R24_A03  4.926699  0.348195   5.399867   1.947112
3  20251114-EBT-T0-T3  R24_A04  5.883694  0.458399   5.104564   1.865928
4  20251114-EBT-T0-T3  R24_A05  4.883356  0.451981   5.288492   1.887264 


=== Archivo generado ===
ysi_feeding_plan.csv

Ejemplo:

    PlateSequenceName   WellId   Glucose   Lactate  Glutamine  Glutamate  \
0  20251114-EBT-T0-T3  R24_A01  5.525105  0.406734   5.203998   2.037303   
1  20251114-EBT-T0-T3  R24_A02  5.728371  0.465841   5.241306   1.911974   
2  20251114-EBT-T0-T3  R24_A03  4.926699  0.348195   5.399867   1.947112   
3  20251114-EBT-T0-T3  R24_A04  5.883694  0.458399   5.104564   1.865928   
4  20251114-EBT-T0-T3  R24_A05  4.8